In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
import minsearch

C:\Users\crab\py2\llm-zoomcamp\cohorts\2025\01-intro\Note\minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = "The course has already started, can I still join"

In [9]:
index.fit(documents)

In [10]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)

In [11]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequen

The result generated consists of other course like machine-learning-zoomcamp, now we make it only retrieve from data-enginerring-zoomcamp

In [12]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [14]:
from openai import OpenAI

In [15]:
import os
from dotenv import load_dotenv

load_dotenv ()

api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:4])

sk-p


In [16]:
client = OpenAI(api_key = api_key)

In [17]:
q

'The course has already started, can I still join'

In [18]:
responses = client.chat.completions.create(
    model = 'gpt-4.1-nano',
    messages = [{"role": "user", "content" : q}]
)

In [19]:
responses.choices[0].message.content

'It depends on the course policies. Please check with the course administrator or contact the instructor to see if late enrollment is allowed. They can provide you with the necessary information and guidance on how to join the course at this stage.'

In [20]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip() 

In [21]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [22]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

In [23]:
prompt = prompt_template.format(question=q, context=context).strip()

In [24]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: The course has already started, can I still join

CONTEXT: section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Reg

In [25]:
responses = client.chat.completions.create(
    model = 'gpt-4.1-nano',
    messages = [{"role": "user", "content" :  prompt}]
)

responses.choices[0].message.content

'YES, you can still join the course even after it has started.'

In [26]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        boost_dict = boost,
        num_results = 5
    )
    
    return results

In [27]:
query = ("How do i run kafka")
results = search(query)

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 

    QUESTION: {question}

    CONTEXT: {context}
    """.strip() 
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [29]:
prompt = build_prompt(query, results)

In [30]:
def llm(prompt):
    responses = client.chat.completions.create(
    model = 'gpt-4.1-nano',
    messages = [{"role": "user", "content" :prompt}]
    )
    
    return responses.choices[0].message.content

In [31]:
query = ("how do i run kafka")

def rag(query):
    results = search(query)
    prompt =  build_prompt(query, results)
    answer = llm(prompt)
    
    return answer

In [32]:
rag(query)

'To run Kafka, you should refer to the techniques described in Module 6: streaming with kafka. Specifically, for Java Kafka, you need to run your producer, consumer, or streams classes in the terminal from the project directory by executing a command like:\n\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nThis command compiles and runs the Kafka producer. Make sure to replace `<jar_name>` with the actual jar file name.'

In [33]:
rag("how do i enroll the course")

'To enroll in the course, you need to register before it starts using the provided registration link.'

In [34]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [35]:
from elasticsearch import Elasticsearch

In [36]:
es_client = Elasticsearch("http://localhost:9200")

In [37]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_setting)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/i-TB69yHS_62aVVmBhTPzg] already exists')

In [38]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [39]:
from tqdm.auto import tqdm

In [40]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [46]:
query = "I just discoverd this course, can I still join this course"

In [54]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [55]:
response = es_client.search(index=index_name, body = search_query)

In [56]:
results_docs = []
for hit in response['hits']['hits']:
    results_docs.append(hit)

In [57]:
results_docs

[{'_index': 'course-questions',
  '_id': 't-UnX5cBQrurkujsxBbP',
  '_score': 80.79293,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '3uUnX5cBQrurkujs0BYs',
  '_score': 58.86029,
  '_source': {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou

In [63]:
def elastic_search(query):
        search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
        
        response = es_client.search(index=index_name, body = search_query)

        results_docs = []
        
        for hit in response['hits']['hits']:
            results_docs.append(hit)

        return results_docs

In [64]:
elastic_search("how do i run kafka")

[{'_index': 'course-questions',
  '_id': 'UeUoX5cBQrurkujsKBgf',
  '_score': 30.447786,
  '_source': {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
   'section': 'Workshop 1 - dlthub',
   'question': 'How do I install the necessary dependencies to run the code?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'OuUoX5cBQrurkujsIhh6',
  '_score': 26.374605,
  '_source': {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
   'section': 'Module 6: streaming with kafka',
   'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-

In [80]:
def build_prompt_es(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 

    QUESTION: {question}

    CONTEXT: {context}
    """.strip() 
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc[ '_source']['section']}\nquestion:{doc[ '_source']['question']}\nanswer:{doc[ '_source']['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [81]:
query = ("how do i run kafka")

def rag(query):
    results = elastic_search(query)
    prompt =  build_prompt_es(query, results)
    answer = llm(prompt)
    
    return answer

In [82]:
rag(query)

'To run Kafka, you need to create a virtual environment and install the required packages. Here are the steps:\n\n1. Create a virtual environment (run only once):\n   ```\n   python -m venv env\n   ```\n2. Activate the virtual environment:\n   - On MacOS/Linux:\n     ```\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```\n     env\\Scripts\\activate\n     ```\n3. Install the required packages from the requirements.txt file:\n   ```\n   pip install -r ../requirements.txt\n   ```\n\nAfter setting up the environment, you can run your Kafka producer or consumer scripts within this environment. \n\n**Note:** Ensure that all Kafka components (like Kafka server and zookeeper) are running according to your setup instructions before executing the scripts.'